In [1]:
import pandas as pd
import numpy as np
from math import pi

# Load the Excel file
excel_path = r"包裝測試.xlsx"  # Replace with your file path
df = pd.read_excel(excel_path, sheet_name='SDS PAN')

# Extract header rows
box_codes = df.iloc[0, 1:].values
box_sizes = df.iloc[1, 1:].values
screw_sizes = df.iloc[2:, 0].values

# Initialize list for cleaned data
clean_data = []

# Helper function to classify codes
def FIND_BOX(code_str):
    s_code, n_code, other_codes = None, None, []
    for part in str(code_str).split('\n'):
        if part.startswith('S') and not s_code:
            s_code = part
        elif part.startswith('N') and not n_code:
            n_code = part
        else:
            other_codes.append(part)
    other_1 = other_codes[0] if len(other_codes) > 0 else None
    other_2 = other_codes[1] if len(other_codes) > 1 else None
    return s_code, n_code, other_1, other_2
    
        # Process the data
for i, screw_size in enumerate(screw_sizes):
    for j, cell in enumerate(df.iloc[i + 2, 1:]):  # Offset by 2 for header rows
        if pd.notna(cell):
            value = str(cell)
            quantity = value.split('-')[0]  # Only extract quantity

            # Sort codes into proper columns
            code1, code2, code3, code4 = FIND_BOX(box_codes[j])

            clean_data.append({
                "Screw Size": screw_size,
                "Quantity": int(quantity),
                "S Box": code1,
                "N Box": code2,
                "Other Box 1": code3,
                "Other Box 1": code4,
                "Box Size": box_sizes[j],
            })

# Convert to DataFrame
df_clean = pd.DataFrame(clean_data)


In [2]:
# Get Screw Volume

head_info = pd.read_excel(r"head size.xlsx")
split_size = df_clean["Screw Size"].str.lower().str.split('x', expand=True)
df_clean['thread_dia'] = split_size[0].astype(float)
df_clean['thread_len'] = split_size[1].astype(float)

# Function to look up head_dia and head_height for a given diameter
def get_head_volume(dia):
        row = head_info.loc[head_info['size'] == dia]
        if not row.empty:
            head_dia = row.iloc[0]['head_dia']
            head_height = row.iloc[0]['head_height']
            return pi * (head_dia / 2) ** 2 * head_height
        return 0  # Return 0 if no matching head info

    # Compute volumes
df_clean['head_dia'] = df_clean['thread_dia'].apply(lambda x: head_info.loc[head_info['size'] == x, 'head_dia'].iloc[0] if not head_info.loc[head_info['size'] == x].empty else None)
df_clean['head_volume'] = df_clean['thread_dia'].apply(get_head_volume)
df_clean['thread_volume'] = pi * (df_clean['thread_dia'] / 2) ** 2 * df_clean['thread_len']
df_clean["screw_volume"] = df_clean['head_volume'] + df_clean['thread_volume']

In [5]:
#Get Emptiness Ratio

split_box = df_clean["Box Size"].str.lower().str.split('x',expand=True)
df_clean['box_volume'] = split_box[0].astype(float)*split_box[1].astype(float)*split_box[2].astype(float)
df_clean["packing_volume"] = df_clean["screw_volume"]*df_clean["Quantity"]
df_clean['emptiness'] = (1-(df_clean['box_volume']-df_clean["packing_volume"])/df_clean['box_volume'])*100
df_clean["dia:len"] = df_clean['head_dia']/df_clean['thread_len']
df_clean.to_excel("train_data.xlsx")

,Screw Size,Quantity,S Box,N Box,Other Box 1,Box Size,thread_dia,thread_len,head_dia,head_volume,thread_volume,screw_volume,box_volume,packing_volume,emptiness,dia:len
0,2.9x13,1000,S2,None,K01,90x68x55,2.9,13.0,5.40,38.933758,85.867581,124.801339,336600.0,124801.338960,37.077047,0.415385
1,3.5x9.5,1000,S2,None,K01,90x68x55,3.5,9.5,6.70,74.038699,91.400711,165.439411,336600.0,165439.410731,49.150152,0.705263
2,3.5x9.5,1000,S3,N1A,K2,95x90x60,3.5,9.5,6.70,74.038699,91.400711,165.439411,513000.0,165439.410731,32.249398,0.705263
3,3.5x9.5,1000,S4,None,B2,135x90x60,3.5,9.5,6.70,74.038699,91.400711,165.439411,729000.0,165439.410731,22.694021,0.705263
4,3.5x13,1000,S3,N1A,K2,95x90x60,3.5,13.0,6.70,74.038699,125.074658,199.113357,513000.0,199113.356986,38.813520,0.515385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,6.3x16,500,S5,N4A,K3,135x90x90,6.3,16.0,12.25,447.863485,498.759250,946.622735,1093500.0,473311.367442,43.284076,0.765625
77,6.3x19,500,S6,N5A,K4,180x90x90,6.3,19.0,12.25,447.863485,592.276609,1040.140094,1458000.0,520070.047100,35.670099,0.644737
78,6.3x25,500,S6,N5A,K4,180x90x90,6.3,25.0,12.25,447.863485,779.311328,1227.174813,1458000.0,613587.406416,42.084184,0.490000
79,6.3x32,500,S7,None,None,190x130x90,6.3,32.0,12.25,447.863485,997.518499,1445.381985,2223000.0,722690.992284,32.509716,0.382812


In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Function to calculate screw volume (in mm³)
def calculate_screw_volume(head_diameter, head_height, shank_diameter, thread_length, head_type='pan'):
    # Shank volume (cylinder)
    shank_volume = np.pi * (shank_diameter / 2) ** 2 * thread_length
    
    # Head volume (approximate based on head type)
    if head_type.lower() == 'pan':
        head_volume = np.pi * (head_diameter / 2) ** 2 * head_height
    elif head_type.lower() == 'flat':
        head_volume = 0.5 * np.pi * (head_diameter / 2) ** 2 * head_height
    elif head_type.lower() == 'round':
        head_volume = (2/3) * np.pi * (head_diameter / 2) ** 3
    else:
        head_volume = 0
    
    return shank_volume + head_volume

# Function to suggest box size and return three outputs
def suggest_box_size(head_diameter, head_height, shank_diameter, thread_length, head_type, quantity, box_table, model, scaler):
    # Calculate screw volume
    screw_volume = calculate_screw_volume(head_diameter, head_height, shank_diameter, thread_length, head_type)
    
    # Calculate packing volume
    packing_volume = screw_volume * quantity
    
    # Prepare input features
    x_new = np.array([[packing_volume, thread_length, head_diameter]])
    x_new_scaled = scaler.transform(x_new)
    
    # Predict emptiness
    predicted_emptiness = float(model.predict(x_new_scaled, verbose=0)[0, 0])
    predicted_emptiness = max(min(predicted_emptiness, 100), 0)  # Bound between 0 and 100
    
    # Calculate required box volume
    required_volume = packing_volume / (predicted_emptiness / 100)
    
    # Find closest box volume
    available_volumes = box_table['box_volume'].values
    closest_volume = min(available_volumes, key=lambda v: abs(v - required_volume))
    
    # Get box details
    box_info = box_table[box_table['box_volume'] == closest_volume][['S Box', 'Box Size']].iloc[0]
    
    # Check if box dimensions accommodate screw length
    box_dims = [float(x) for x in box_info['Box Size'].lower().split('x')]
    if min(box_dims) <= thread_length:
        raise ValueError(f"No suitable box found: smallest dimension {min(box_dims)} mm is less than screw length {thread_length} mm")
    
    return {
        'suggest box type': box_info['S Box'],
        'emptiness': predicted_emptiness,
        'predict volume': required_volume
    }

# Create box_table
box_table = df_clean[['S Box', 'Box Size', 'box_volume']].drop_duplicates().reset_index(drop=True)

# Prepare data for neural network
X = df_clean[['packing_volume', 'thread_len', 'head_dia']].values
y = df_clean['emptiness'].values

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Build neural network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=16, verbose=0)

# Example screw inputs
head_diameter = 6.7   # mm
head_height = 2       # mm (assumed)
shank_diameter = 3.5  # mm
thread_length = 9.5   # mm
head_type = 'pan'     # Assumed
quantity = 1000

# Suggest box size
result = suggest_box_size(head_diameter, head_height, shank_diameter, thread_length, head_type, quantity, box_table, model, scaler)

# Output results
print("Suggested Box Type:", result['suggest box type'])
print("Emptiness:", round(result['emptiness'], 2), "%")
print("Predicted Volume:", round(result['predict volume'], 2), "mm³")

C:\Users\wesley\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Suggested Box Type: S2
Emptiness: 40.59 %
Predicted Volume: 398876.06 mm³


In [12]:
DATABASE = r"Z:\跨部門\共用資料夾\C. 業務部\詢價統計DB\QUOTATION_DATABASE.db"

import sqlite3
def SEARCH_THROUGH_RFQ():
    with sqlite3.connect(DATABASE) as connection:
        cursor = connection.cursor()
        query = f"SELECT * FROM CUSTOMER_PRODUCT_SUMMARY WHERE CUSTOMER_CODE = 'C01900' AND DESCRIPTION \
                LIKE '%鑽尾螺絲%'"
        df = pd.read_sql_query(query, connection)
        
        # Check if quotation exists
        if df.empty:
            return None  # Return None instead of empty DataFrame
    return df
quote_df = SEARCH_THROUGH_RFQ()
quote_df[["SIZE", "M_BOX", "BOX_TYPE", "DESCRIPTION"]]

,SIZE,M_BOX,BOX_TYPE,DESCRIPTION
0,3.9x32,1.0,S6,"品名：鑽尾螺絲 六角華司頭(凹頭) 凹切, BSD 3號尾, 滲碳C.D-2(滲碳依據實際心..."
1,4.8x25,1.0,S7,"品名：鑽尾螺絲 六角華司頭(凹頭) 凹切, BSD 3號尾, 滲碳C.D-2(滲碳依據實際心..."
2,5.5x19,0.5,S5,"品名：鑽尾螺絲 六角華司頭(凹頭) 凹切, BSD 3號尾, 滲碳C.D-2(滲碳依據實際心..."
3,6.3x25,0.5,S6,"品名：鑽尾螺絲 六角華司頭(凹頭) 凹切, BSD 3號尾, 滲碳C.D-2(滲碳依據實際心..."
4,4.2x16,1.0,S5,"品名：鑽尾螺絲 盤頭, BSD 3號尾, 滲碳C.D-2(滲碳依據實際心部硬度+30HV),..."
...,...,...,...,...
1198,5.5x25,0.5,S5,品名：鑽尾螺絲 盤頭 十字孔 / 熱處理 鍍鋅CR3+均3U / 牛皮無印刷S盒小包裝(無外...
1199,5.5x32,0.5,S6,品名：鑽尾螺絲 盤頭 十字孔 / 熱處理 鍍鋅CR3+均3U / 牛皮無印刷S盒小包裝(無外...
1200,4.2x13,1.0,S3,品名：鑽尾螺絲 平頭 十字孔 熱處理 / 鍍鋅CR3+均3U / 牛皮無印刷S盒小包裝(...
1201,4.2x16,1.0,S4,品名：鑽尾螺絲 平頭 十字孔 熱處理 / 鍍鋅CR3+均3U / 牛皮無印刷S盒小包裝(...


In [16]:
PAN_HEAD_DATA = quote_df[quote_df["DESCRIPTION"].str.contains("盤頭")][["SIZE", "M_BOX", 
                                                                      "BOX_TYPE", "DESCRIPTION"]]
PAN_HEAD_DATA["M_BOX"] = PAN_HEAD_DATA["M_BOX"]*1000
PAN_HEAD_DATA 

,SIZE,M_BOX,BOX_TYPE,DESCRIPTION
4,4.2x16,1000.0,S5,"品名：鑽尾螺絲 盤頭, BSD 3號尾, 滲碳C.D-2(滲碳依據實際心部硬度+30HV),..."
5,4.8x16,1000.0,S5,"品名：鑽尾螺絲 盤頭, BSD 3號尾, 滲碳C.D-2(滲碳依據實際心部硬度+30HV),..."
6,6.3x80,250.0,S7,"品名：鑽尾螺絲 盤頭, BSD 3號尾, 滲碳C.D-2(滲碳依據實際心部硬度+30HV),..."
15,3.5x13,1000.0,S3,"品名：鑽尾螺絲 梅花孔 盤頭, BSD 3號尾, 滲碳C.D-2(滲碳依據實際心部硬度+30..."
16,3.5x16,1000.0,S4,"品名：鑽尾螺絲 梅花孔 盤頭, BSD 3號尾, 滲碳C.D-2(滲碳依據實際心部硬度+30..."
...,...,...,...,...
1195,3.5x13,1000.0,S3,品名：鑽尾螺絲 盤頭 十字孔 / 熱處理 鍍鋅CR3+均3U / 牛皮無印刷S盒小包裝(無外...
1196,3.9x13,1000.0,S4,品名：鑽尾螺絲 盤頭 十字孔 / 熱處理 鍍鋅CR3+均3U / 牛皮無印刷S盒小包裝(無外...
1197,4.8x60,250.0,S6,品名：鑽尾螺絲 盤頭 十字孔 / 熱處理 鍍鋅CR3+均3U / 牛皮無印刷S盒小包裝(無外...
1198,5.5x25,500.0,S5,品名：鑽尾螺絲 盤頭 十字孔 / 熱處理 鍍鋅CR3+均3U / 牛皮無印刷S盒小包裝(無外...


In [4]:
int(1.2)

1